In [2]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic"

## Resampled Louvain cluster GSEA

In [7]:
##########----- Parameters-----##########
analysis_name <- c("0", "resampled")
genome <- "mm"
#########################################
analysis_subdir <- paste(analysis_name[1], "Scanpy_out", analysis_name[2], sep="_")
sum.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "3_GSEA")
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)


###----- Select gene signature
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm')
gs.pattern <- paste("_", genome, "_sigs.csv", sep="")
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
gs.files

[1] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD4_mm_sigs.csv"               
[2] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD8_mm_sigs.csv"               
[3] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Exp391_CD8_mm_sigs.csv"        
[4] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Wolski_curated_CD8_mm_sigs.csv"

In [9]:
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/202009_CD8')
gs.pattern <- "_signatures_mouse.csv"
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
gs.files

[1] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_Formatted_csv/202009_CD8/2016_SCIENCE_Mackay_signatures_mouse.csv"

In [10]:
z.use.keys <- c('louvain', 'cell_type', 'louvain_celltype') 

#####---------- For each classification method
for (z.use.key in z.use.keys) {
    z.file.name <- paste(z.use.key, 'mean_scaled_expr_z.csv', sep="_")
    z.file <- file.path(base_dir, '1_SCANPY', analysis_subdir, "1_avg_expr", z.file.name)
    z.tb <- read_csv(z.file)
    
    #####---------- For each signature collection, calculate GSEA & summarize
    for (gs.file in gs.files) {
        # Create sub directory for signature group
        gs.file.base <- gsub(gs.pattern, "",basename(gs.file))

        sep.dir <- paste(z.use.key, "sep", sep="_")
        wk.dir <- file.path(sum.dir, sep.dir, gs.file.base)
        dir.create(wk.dir, showWarnings = FALSE, recursive=TRUE)
        setwd(wk.dir)

        ###----- Run GSEA
        for (i in tail(colnames(z.tb), -1)){
            useGroup <- i
            outName <- i
            gsea <- tryCatch({
                    GSEA_analysis(z.file, useGroup, outName, gs.file, 1)
                }, error=function(cond) {
                    message(paste(z.use.key, gs.file, i, sep='; '))
                })
        }

        ###----- Summarize
        setwd(sum.dir)

        gsea.sum.name <- paste(z.use.key, "gsea", gs.file.base, sep="_")

        gs.all.tb <- read_csv(gs.file)
        path.order <- str_sort(unique(gs.all.tb$gs_name), numeric = TRUE)
        gs.file.ext <- basename(gs.file)
        gs.file.ext <- paste("---", gs.file.ext, sep="")

        comp.order <- sort(tail(colnames(z.tb), -1))

        gsea.files <- list.files(path = wk.dir, pattern = "*.csv", full.names = TRUE)
        gsea.file.base <- gsub(gs.file.ext, "", basename(gsea.files))
        comp.order <- intersect(comp.order, gsea.file.base)
        comp.order <- str_sort(comp.order, numeric = TRUE)
        print(comp.order)

        sum_df <- GSEA_sum(gsea.files, gsea.sum.name, 40, 20, TRUE, comp.order, TRUE, path.order)
    }
}

Parsed with column specification:
cols(
  gene_name = col_character(),
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double(),
  `10` = col_double()
)

Parsed with column specification:
cols(
  gene_name = col_character(),
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double(),
  `10` = col_double()
)

Warning message:
“Unknown columns: `gene_names`”
Parsed with column specification:
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)

preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...

Parsed with column specification:
cols(
  g

leading edge analysis...

done...

Parsed with column specification:
cols(
  gene_name = col_character(),
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double(),
  `10` = col_double()
)

Warning message:
“Unknown columns: `gene_names`”
Parsed with column specification:
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)

preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...

Parsed with column specification:
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)



 [1] "0"  "1"  "2"  "3"  "4"  "5"  "6"  "7"  "8"  "9"  "10"


Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_ed

# A tibble: 6 x 6
  comparison pathway          NES     padj leadingEdge_signal mlog10padj
  <chr>      <chr>          <dbl>    <dbl>              <dbl>      <dbl>
1 0          TEM_vs_TCM-TRM  1.81 0.000475                 25       3.32
2 0          TCM_vs_TEM-TRM -2.19 0.000475                 43       3.32
3 0          TRM_vs_TCM-TEM -1.48 0.00101                  24       2.99
4 1          TRM_vs_TCM-TEM  1.65 0.000300                 15       3.52
5 1          TCM_vs_TEM-TRM -2.71 0.000926                 46       3.03
6 1          TEM_vs_TCM-TRM  1.49 0.0334                   25       1.48


...

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edg

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edg

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)

Parsed with column specification:
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edg

# A tibble: 6 x 6
  comparison pathway           NES    padj leadingEdge_signal mlog10padj
  <chr>      <chr>           <dbl>   <dbl>              <dbl>      <dbl>
1 0_A5P      TCM_vs_TEM-TRM -1.69  0.00176                 33      2.75 
2 0_A5P      TEM_vs_TCM-TRM  1.18  0.305                   25      0.516
3 0_A5P      TRM_vs_TCM-TEM  0.946 0.679                   18      0.168
4 0_A5T      TCM_vs_TEM-TRM -1.46  0.0445                  40      1.35 
5 0_A5T      TRM_vs_TCM-TEM  1.17  0.119                   18      0.923
6 0_A5T      TEM_vs_TCM-TRM -1.01  0.423                   22      0.373
